This notebook is supposed to trace the chapter 4 for fastbook.

However it is not easy for multilabel classification, using trivial model, therefore ill have to read chapter 6.

SO will be back.`

In [1]:
from fastai.vision.all import *
import pandas as pd

### Getting the data

In [2]:
path = Path('../input/sign-language-mnist')
path.ls()

(#7) [Path('../input/sign-language-mnist/sign_mnist_test.csv'),Path('../input/sign-language-mnist/sign_mnist_test'),Path('../input/sign-language-mnist/sign_mnist_train'),Path('../input/sign-language-mnist/amer_sign2.png'),Path('../input/sign-language-mnist/amer_sign3.png'),Path('../input/sign-language-mnist/sign_mnist_train.csv'),Path('../input/sign-language-mnist/american_sign_language.PNG')]

In [3]:
TRAIN = path/'sign_mnist_train'/'sign_mnist_train.csv'

In [4]:
TEST = path/'sign_mnist_test'/'sign_mnist_test.csv'

In [5]:
train_df = pd.read_csv(TRAIN)

In [6]:
train_df.head()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,3,107,118,127,134,139,143,146,150,153,...,207,207,207,207,206,206,206,204,203,202
1,6,155,157,156,156,156,157,156,158,158,...,69,149,128,87,94,163,175,103,135,149
2,2,187,188,188,187,187,186,187,188,187,...,202,201,200,199,198,199,198,195,194,195
3,2,211,211,212,212,211,210,211,210,210,...,235,234,233,231,230,226,225,222,229,163
4,13,164,167,170,172,176,179,180,184,185,...,92,105,105,108,133,163,157,163,164,179


In [7]:
train_df.iloc[0]

label         3
pixel1      107
pixel2      118
pixel3      127
pixel4      134
           ... 
pixel780    206
pixel781    206
pixel782    204
pixel783    203
pixel784    202
Name: 0, Length: 785, dtype: int64

In [8]:
#test
#pd.DataFrame(tensor(train_df.iloc[0])).style.set_properties(**{'font-size':'6pt'}).background_gradient('Greys')

getting labels and data from dataframe: https://www.kaggle.com/fanbyprinciple/kannada-mnist-fastai-pixel-to-image-conv

### Trying to use datablock api

In [9]:
import numpy as np

In [10]:
def get_y(t_df):
    return t_df['label'].to_numpy()

get_y(train_df)

# this works for y

array([ 3,  6,  2, ..., 18, 17, 23])

In [11]:
def get_x(t_df):
    return t_df.drop('label',axis=1).to_numpy()

get_x(train_df)

array([[107, 118, 127, ..., 204, 203, 202],
       [155, 157, 156, ..., 103, 135, 149],
       [187, 188, 188, ..., 195, 194, 195],
       ...,
       [174, 174, 174, ..., 202, 200, 200],
       [177, 181, 184, ...,  64,  87,  93],
       [179, 180, 180, ..., 205, 209, 215]])

### Tensor from dataframe


In [12]:
# note to self
# can this be done without using the to_numpy()


# using pytorch tensor function
# hint: https://discuss.pytorch.org/t/how-to-convert-array-to-tensor/28809/7


def get_data_and_labels(data_frame, label):
    labels = data_frame[label].to_numpy()
    data = data_frame.drop([label], axis=1).to_numpy(dtype=np.float32).reshape((data_frame.shape[0], 28,28))
    data = np.expand_dims(data, axis=1)
    return data, labels

In [13]:
train_data, train_labels = get_data_and_labels(train_df, 'label')

In [14]:
# https://www.kaggle.com/raghuramr/mnist-by-beginner-with-fastai-resnet34

def get_images(df):
    IMG_WIDTH = 28
    IMG_HEIGHT = 28
    
    df['img'] = df[df.columns[df.columns.str.startswith('pixel')]].apply(
        lambda x : PILImage(Image.fromarray(np.array(x.values).reshape((IMG_WIDTH, IMG_HEIGHT)).astype(np.uint8))),axis=1)
    
    return df[df.columns[[not x for x in df.columns.str.startswith('pixel')]]]

In [15]:
train_im = get_images(train_df)

In [16]:
def new_get_x(r): return r['img']
def new_get_y(r): return r['label']

In [17]:
hands = DataBlock(blocks=(ImageBlock, CategoryBlock),
                  get_x=new_get_x,
                  get_y=new_get_y,
                  splitter=RandomSplitter(seed=42),
                  item_tfms = Resize(460),
                  batch_tfms = aug_transforms(size=224, min_scale=0.75))

try:
    hand_dls = hands.dataloaders(train_im) # hand_dls, get it ?
except Exception as e:
    print(e)

In [ ]:
learn = cnn_learner(hand_dls, resnet18, metrics=error_rate).to_fp16()
learn.fine_tune(2)

Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to /root/.cache/torch/hub/checkpoints/resnet18-5c106cde.pth


In [ ]:
test_df = pd.read_csv(TEST)
test_df['ImageId'] = test_df.index+1
test_df = get_images(test_df)
test_df.head()

In [ ]:
dl = learn.dls.test_dl(list(test_df['img']))
inp, preds, _, dec_preds = learn.get_preds(dl=dl, with_input=True,with_decoded=True)
test_df['Label'] = dec_preds

In [ ]:
submission = test_df[['ImageId', 'Label']]
submission.to_csv("submission.csv")

In [ ]:
# train_data[:5],train_labels[:5]

In [ ]:
# show_image(train_data[7][0])
# # 
# # image of 28 by 28

In [ ]:
# len(train_data), len(train_labels), train_data.shape, train_labels.shape

In [ ]:
# plt.imshow(train_data[7,0])

In [ ]:
# train_labels[7]

# # I was initially confused why not 3
# # then I realised its sign language

### Validation set

In [ ]:
# np.random.seed(42)

# # note to self
# # use a splitter next time

# # creating a validation set
# ran_10_pct_idx = (np.random.random_sample(train_labels.shape)) < .1


# train_90_labels = train_labels[np.invert(ran_10_pct_idx)]
# train_90_data = train_data[np.invert(ran_10_pct_idx)]

# valid_10_labels = train_labels[ran_10_pct_idx]
# valid_10_data = train_data[ran_10_pct_idx]

### changing the shape of images

In [ ]:
# train_x = tensor(train_90_data).view(-1,28*28)

# # to self
# # calling tensor solves the error "Type must be a sub-type of ndarray type"

In [ ]:
# train_x.shape

In [ ]:
# train_y = tensor(train_90_labels).unsqueeze(1)
# #train_y = tensor(train_90_labels)

In [ ]:
# train_y.shape

In [ ]:
# val_x = tensor(valid_10_data).view(-1,28*28)
# val_y = tensor(valid_10_labels).unsqueeze(1)
# #val_y = tensor(valid_10_labels)
# val_x.shape, val_y.shape

In [ ]:
# train_y

### Creating a simple datasets

In [ ]:
# train_dset = list(zip(train_x, train_y))
# valid_dset = list(zip(val_x, val_y))

In [ ]:
# len(train_dset),len(valid_dset)

### Feeding dataset into dataloader

In [ ]:
# dl = DataLoader(train_dset, batch_size=256)
# val_dl = DataLoader( valid_dset, batch_size=256)

In [ ]:
# # lets test

# xb, yb = first(dl)
# xb.shape, yb.shape

In [ ]:
# v_xb, v_yb = first(val_dl)
# v_xb.shape, v_yb.shape

### Starting simple : creating our own model
which is not so simple tho

In [ ]:
# # for weights and bias inititalisation
# def init_params(size, std=1.0):
#     return (torch.randn(size) * std).requires_grad_()

In [ ]:
# weights = init_params((28*28,1))

In [ ]:
# bias = init_params(1)

In [ ]:
# # trying on sample
# (train_x[7]*weights.T).sum() + bias

In [ ]:
# # defining the prediction function
# def linear1(xb):
#     return xb@weights+ bias

lets do it in a small batch

In [ ]:
# x, y =  train_dset[0]

In [ ]:
# linear1(x), y

# # there is no way that the linear1(x) value is equal to 3

In [ ]:
# inference on simple prediction function
# preds = linear1(train_x)

In [ ]:
# preds

Now here i swhere things diverge
Since instead of zero and one is spread of entire numerical range we will have to adjust our function for multi inputs rather than zero and one

In [ ]:
# corrects = (preds).float() == train_y

In [ ]:
# corrects

In [ ]:
# corrects.float().mean().item()
# # woah thats our accuracy right now!

so yes, direct equal to is not available. We need to find another parameter for loss.
from : https://medium.com/@thevatsalsaglani/training-and-deploying-a-multi-label-image-classifier-using-pytorch-flask-reactjs-and-firebase-c39c96f9c427

In [ ]:
# def pred_acc(original, predicted):
#     return torch.round(predicted).eq(original).sum().numpy()/len(original)

In [ ]:
# pred_acc(preds.float(), train_y.float())

We found that simple linear equation won't hold for working with fastai

In [ ]:
# # will manually changing weights help ?
# weights[2] *= 1.002
# preds = linear1(train_x)
# corrects = (preds>0.0).float() == train_y
# corrects.float().mean().item()
# # apparently nothing

In [ ]:
# def mnist_loss(predictions, targets):
#     predictions = predictions.sigmoid()
#     return torch.where(targets == 1, 1-predictions, predictions).mean()

In [ ]:
# loss = mnist_loss(preds, train_y)
# loss

In [ ]:
# def calc_grad(xb, yb, model):
#     preds = model(xb)
#     loss = mnist_loss(preds, yb)
#     loss.backward()

In [ ]:
# def train_epoch(model, lr, params,dl):
#     for xb, yb in dl:
#         calc_grad(xb, yb, model)
#     # model is a function in this case linear1
#         for p in params:
#             p.data -= p.grad*lr
#             p.grad.zero_()

In [ ]:
# def batch_accuracy(xb, yb):
#     preds = xb.sigmoid()
#     correct = (preds>0.5) == yb
#     return correct.float().mean()

In [ ]:
# def validate_epoch(model):
#     accs = [batch_accuracy(model(xb),yb) for xb, yb in valid_dl]
#     return round(torch.stack(accs).mean().item(), 4)

In [ ]:
# lr = 1.
# for i in range(30):
#     train_epoch(linear1, lr, params, dl)
#     print(validate_epoch(linear1), end=' ')

In [ ]:
# # using dataloaders class

# dls = DataLoaders(dl, val_dl)

### Creating a custom architecture
https://www.kaggle.com/fanbyprinciple/kannada-mnist-fastai-pixel-to-image-conv

In [ ]:
## TBD

### Learner class

In [ ]:
# learn = Learner(dls, nn.Linear(28*28,1), loss_func=F.cross_entropy, metrics=accuracy)

In [ ]:
# learn.fit_one_cycle(42)